In [5]:
# import libraries
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

# Loading the dataset
mnist_dataset, mnist_info = tfds.load('mnist', with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\annes\tensorflow_datasets\mnist\3.0.1.incompleteRY16Y8\mnist-train.tfrecord*...:   0%|     …

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\annes\tensorflow_datasets\mnist\3.0.1.incompleteRY16Y8\mnist-test.tfrecord*...:   0%|      …

Dataset mnist downloaded and prepared to C:\Users\annes\tensorflow_datasets\mnist\3.0.1. Subsequent calls will reuse this data.


# Preprocessing the dataset

In [23]:
# splitting the data
mnist_train, mnist_test= mnist_dataset['train'], mnist_dataset['test']
# number of validation samples
num_validation = 0.1*mnist_info.splits['train'].num_examples
num_validation = tf.cast(num_validation, tf.int64) # converts it to int64
# number of test samples
num_test = mnist_info.splits['test'].num_examples
num_test = tf.cast(num_test, tf.int64)

# scale the data to the range [0,1]
def scale(image, label):
    image = tf.cast(image, tf.float64)
    image /= 255.
    return image, label

scaled_train_nd_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [24]:
# buffer size
buffer_size = 10000
# shuffle the data
shuffled_train_nd_validation_data = scaled_train_nd_validation_data.shuffle(buffer_size)
# split the shuffled train and validation data
train_data = shuffled_train_nd_validation_data.skip(num_validation)
validation_data = shuffled_train_nd_validation_data.take(num_validation)
# batch size
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)# combines the consecutive elements into batches
# we don't need to batch the validation and the test data
validation_data = validation_data.batch(num_validation)
test_data = test_data.batch(num_test)

# extract and convert the validation inputs and targets
validation_inputs, validation_targets = next(iter(validation_data)) #loads the next element of an iterable object ( with the iter fct )

# Model

## Outline the model

In [40]:
input_size = 784
output_size = 10
hidden_layer_size = 100

# building the model
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)), # transform tensor into a vector
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), #build each consecutive layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # the second layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # the third layer
    tf.keras.layers.Dense(output_size, activation='softmax') # the last layer
])

model

## Choose the optimizer and the loss function

In [42]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## Training the model

In [43]:
NUM_EPOCHS = 10
# fit the model
model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/10
540/540 - 4s - loss: 0.3264 - accuracy: 0.9045 - val_loss: 0.1548 - val_accuracy: 0.9547 - 4s/epoch - 8ms/step
Epoch 2/10
540/540 - 2s - loss: 0.1298 - accuracy: 0.9603 - val_loss: 0.1012 - val_accuracy: 0.9702 - 2s/epoch - 4ms/step
Epoch 3/10
540/540 - 3s - loss: 0.0903 - accuracy: 0.9721 - val_loss: 0.0772 - val_accuracy: 0.9793 - 3s/epoch - 5ms/step
Epoch 4/10
540/540 - 3s - loss: 0.0706 - accuracy: 0.9779 - val_loss: 0.0640 - val_accuracy: 0.9788 - 3s/epoch - 5ms/step
Epoch 5/10
540/540 - 3s - loss: 0.0547 - accuracy: 0.9835 - val_loss: 0.0593 - val_accuracy: 0.9833 - 3s/epoch - 5ms/step
Epoch 6/10
540/540 - 3s - loss: 0.0460 - accuracy: 0.9848 - val_loss: 0.0598 - val_accuracy: 0.9805 - 3s/epoch - 5ms/step
Epoch 7/10
540/540 - 3s - loss: 0.0378 - accuracy: 0.9881 - val_loss: 0.0496 - val_accuracy: 0.9837 - 3s/epoch - 5ms/step
Epoch 8/10
540/540 - 3s - loss: 0.0331 - accuracy: 0.9890 - val_loss: 0.0428 - val_accuracy: 0.9857 - 3s/epoch - 5ms/step
Epoch 9/10
540/540 - 3s 

## Test the model

In [46]:
# evaluate the model
test_loss, test_accuracy = model.evaluate(test_data)
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}'.format(test_loss, test_accuracy*100.))

1/1 [==============================] - 0s 199ms/step - loss: 0.0940 - accuracy: 0.9756
Test loss: 0.09. Test accuracy: 97.56
